# Explain basic LSTM from Keras Sequential Model

In [1]:
import shap, keras
import numpy as np
import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 1000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 100
X_train = np.expand_dims(sequence.pad_sequences(X_train, maxlen=max_review_length),axis=2)
X_test = np.expand_dims(sequence.pad_sequences(X_test, maxlen=max_review_length),axis=2)
# create the model
embedding_vector_length = 32
mod = Sequential()
# mod.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
mod.add(LSTM(100, input_shape=(100,1)))
mod.add(Dense(1, activation='sigmoid'))
mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(mod.summary())
# mod.fit(np.expand_dims(X_train[0:5000,],axis=2), y_train[0:5000], epochs=1, batch_size=64)
# Final evaluation of the model
# scores = mod.evaluate(np.expand_dims(X_test,axis=2), y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))
# mod.save('lstm_model.h5')  # creates a HDF5 file 'my_model.h5'

# # Look at things in tensorboard
# session = keras.backend.get_session()
# file_writer = tf.summary.FileWriter('tmp/', session.graph)
# # session.graph.get_operations()
# # tensorboard --logdir /Users/hughchen/Desktop/deepshap/shap/notebooks/tmp/

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
inds = np.random.choice(X_train.shape[0], 3, replace=False)
data = X_train[inds,:]
test_in = X_test[10:11,:,:]
e = shap.DeepExplainer((mod.layers[0].input, mod.layers[-1].input), data)
shap_values = e.shap_values(test_in)
sums = np.array([shap_values[i].sum() for i in range(len(shap_values))])
sess = tf.keras.backend.get_session()
diff = sess.run(mod.layers[-1].input, feed_dict={mod.layers[0].input: test_in})[0,:] - \
    sess.run(mod.layers[-1].input, feed_dict={mod.layers[0].input: data}).mean(0)
assert np.allclose(sums, diff, atol=1e-06), "Sum of SHAP values does not match difference!"